### Imports 

In [1]:
import pandas as pd
from glob import glob
import shutil
import tensorflow as tf
import cv2
from matplotlib import pyplot as plt
import random
import os
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import applications
import re

### Load train data and visualize 

In [2]:
df1 = pd.read_csv('../dataset/Barplot_splitted.csv').drop(columns=['Unnamed: 0'])
df2 = pd.read_csv('../dataset/LinePlot_splitted.csv').drop(columns=['Unnamed: 0'])
df3 = pd.read_csv('../dataset/Scatter_splitted.csv').drop(columns=['Unnamed: 0'])
df = pd.concat([df1,df2,df3]).reset_index().drop(columns=['index'])

In [6]:
#Select a random train plot
random_index = random.randint(0,len(df)-1)
selected_chart = df.iloc[random_index]['graph_dir']
index_plot = re.findall(r'[0-9]+',selected_chart.split('\\')[-1].split('.')[0] )
plot_type = re.findall(r'[A-Za-z]+', selected_chart.split('\\')[-1].split('.')[0] )

In [24]:
d1 = eval(df.iloc[index_plot]['model'][int(index_plot[0])])
d2 = eval(df.iloc[index_plot]['general_figure_info'][int(index_plot[0])])
key_list = ['bbox','bboxes','bar_bboxes']

In [23]:
eval(d1)

[{'name': 'Australia',
  'color': 'm',
  'label': 'Australia',
  'coordinates': [{'x': 549.8703339363788, 'y': 588.2222760751936},
   {'x': 874.9973503693652, 'y': 506.10576935423455},
   {'x': 1200.1243668023517, 'y': 490.6344787434403},
   {'x': 1525.2513832353384, 'y': 448.24649614260244},
   {'x': 1850.3783996683248, 'y': 448.1383506808346},
   {'x': 2175.5054161013113, 'y': 497.9951907313116},
   {'x': 2500.632432534298, 'y': 590.9579536298554}],
  'x': [0, 1, 2, 3, 4, 5, 6],
  'y': [4228628084.90412,
   3463220020.24218,
   3319012100.57908,
   2923913660.29009,
   2922905636.30199,
   3387621306.4762,
   4254127338.90634],
  'bar_bboxes': [{'x': 492.97310606060614,
    'y': 134.5555555555556,
    'w': 113.79445575154523,
    'h': 453.666720519638},
   {'x': 818.1001224935926,
    'y': 134.5555555555556,
    'w': 113.79445575154523,
    'h': 371.55021379867895},
   {'x': 1143.2271389265793,
    'y': 134.5555555555556,
    'w': 113.79445575154523,
    'h': 356.0789231878847},
   {

In [17]:
import json
json.loads(d1)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [23]:
bbox_and_bboxes_data = []
def seek_keys(d, key_list):
    for k, v in d.items():
        if k in key_list:
            #print(k + ": " + str(v))
            bbox_and_bboxes_data.append(v)
        if isinstance(v, dict):
            seek_keys(v, key_list)

In [24]:
seek_keys(d1,key_list) #bar bboxes
seek_keys(d2,key_list)

AttributeError: 'str' object has no attribute 'items'

In [13]:
chart_array = cv2.imread(selected_chart)
y_factor, x_factor = [x/640 for x in chart_array.shape[0:2]]
chart_array = cv2.resize(chart_array, (640,640))
for element in bbox_and_bboxes_data:
    if isinstance(element,dict): 
        image = chart_array
        start_point = (round(element['x']/x_factor), round(chart_array.shape[0]-element['y']/y_factor))
        end_point = (round(element['x']/x_factor + element['w']/x_factor), round((chart_array.shape[0]-element['y']/y_factor)-(element['h']/y_factor)))
        color = (255,0,0)
        thickness = 3
        chart_array = cv2.rectangle(image , start_point, end_point, color, thickness)
    elif isinstance(element,list):
        for sub_element in element: 
            image = chart_array
            start_point = (round(sub_element['x']/x_factor), round(chart_array.shape[0]-sub_element['y']/y_factor))
            try:
                end_point = (round(sub_element['x']/x_factor + sub_element['w']/x_factor), round((chart_array.shape[0]-sub_element['y']/y_factor)-(sub_element['h']/y_factor)))
            except:
                end_point = (round(sub_element['x']/x_factor), round((chart_array.shape[0]-sub_element['y']/y_factor)))
                    
            color = (255,0,0)
            thickness = 3
            chart_array = cv2.rectangle(image , start_point, end_point, color, thickness)

### Data preprocessing - Prepare data for training
We also convert everything to the format that the training loop below expects (e.g., everything converted to tensors, classes converted to one-hot representations, etc.).



In [12]:
df

,model,type,general_figure_info,image_index
0,{'name': 'Percentage of part-time workers in L...,bar_plot,{'title': {'text': 'Percentage of part-time wo...,0
1,"{'name': 'Total debt of Jamaica', 'color': 'k'...",bar_plot,"{'title': {'text': 'Total debt of Jamaica', 'b...",1
2,{'name': 'Arable land in different countries a...,bar_plot,{'title': {'text': 'Arable land in different c...,2
3,{'name': 'Percentage of households with a fema...,bar_plot,{'title': {'text': 'Percentage of households w...,3
4,{'name': 'Percentage of people living with HIV...,bar_plot,{'title': {'text': 'Percentage of people livin...,4
...,...,...,...,...
1107,{'name': 'Measure of Ecuador's business execut...,bar_plot,{'title': {'text': 'Measure of Ecuador's busin...,1107
1108,{'name': 'Percentage of unemployed male labor ...,bar_plot,{'title': {'text': 'Percentage of unemployed m...,1108
1109,{'name': 'Cost of communications and computers...,bar_plot,{'title': {'text': 'Cost of communications and...,1109
1110,{'name': 'Number of neonatal deaths in the yea...,bar_plot,{'title': {'text': 'Number of neonatal deaths ...,1110


In [ ]:
# By convention, our non-background classes start counting at 1.  Given
# that we will be predicting just one class, we will therefore assign it a
# `class id` of 1.

duck_class_id = 1
num_classes = 1

category_index = {duck_class_id: {'id': duck_class_id, 'name': 'rubber_ducky'}}

# Convert class labels to one-hot; convert everything to tensors.
# The `label_id_offset` here shifts all classes by a certain number of indices;
# we do this here so that the model receives one-hot labels where non-background
# classes start counting at the zeroth index.  This is ordinarily just handled
# automatically in our training binaries, but we need to reproduce it here.

label_id_offset = 1
train_image_tensors = []
gt_classes_one_hot_tensors = []
gt_box_tensors = []
for (train_image_np, gt_box_np) in zip(
    train_images_np, gt_boxes):
    train_image_tensors.append(tf.expand_dims(tf.convert_to_tensor(train_image_np, dtype=tf.float32), axis=0))
    gt_box_tensors.append(tf.convert_to_tensor(gt_box_np, dtype=tf.float32))
    zero_indexed_groundtruth_classes = tf.convert_to_tensor(np.ones(shape=[gt_box_np.shape[0]], dtype=np.int32) - label_id_offset)
    gt_classes_one_hot_tensors.append(tf.one_hot(zero_indexed_groundtruth_classes, num_classes))
print('Done prepping data.')

In [28]:
df.iloc[0]['general_figure_info'] #title, x_axis_bboxes, y_axis_bboxes, ticks_bboxes, 

{'title': {'text': 'Percentage of part-time workers in Luxembourg',
  'bbox': {'x': 1154.9027777777778,
   'y': 1517.9999999999995,
   'w': 995.75,
   'h': 42.0}},
 'x_axis': {'tick_labels': {'value': ['1991', '1992', '1993', '1994', '1995'],
   'bboxes': [{'x': 339.94689713224193,
     'y': 73.11111111111116,
     'w': 105.75,
     'h': 42.0},
    {'x': 969.9873374550098, 'y': 73.11111111111116, 'w': 105.625, 'h': 42.0},
    {'x': 1599.8402777777776, 'y': 73.11111111111116, 'w': 105.875, 'h': 42.0},
    {'x': 2229.8807181005454, 'y': 73.11111111111116, 'w': 105.75, 'h': 42.0},
    {'x': 2859.8586584233135,
     'y': 73.11111111111116,
     'w': 105.75,
     'h': 42.0}]},
  'ticks_coordinates': {'value': [0, 1, 2, 3, 4],
   'bboxes': [{'x': 392.82189713224193, 'y': 134.5555555555556},
    {'x': 1022.7998374550098, 'y': 134.5555555555556},
    {'x': 1652.7777777777776, 'y': 134.5555555555556},
    {'x': 2282.7557181005454, 'y': 134.5555555555556},
    {'x': 2912.7336584233135, 'y': 134.